Topic: Challenge Set 1 Subject: Explore MTA turnstile data Date: 04/13/2018 Name: student name Worked with: other students' name Location: sea18_ds10/student_submissions/challenges/01-mta/shaikh_reshama/challenge_set_1_reshama.ipynb

In [225]:
import sys

In [226]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

import pprint

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

import dateutil.parser
from datetime import *

In [227]:
# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 3)

In [228]:
#read data from this folder
df1 = pd.read_csv('turnstile_171007.csv')
df2 = pd.read_csv('turnstile_170902.csv')
df3 = pd.read_csv('turnstile_171021.csv')

In [229]:
#Work with turnstile name as combo
def Col_Con(mta):
    mta['TURNSTILE'] = mta['C/A'] + mta['UNIT'] + mta['SCP']
    mta.drop('C/A', axis=1, inplace = True)
    mta.drop('UNIT', axis=1,inplace = True)
    mta.drop('SCP', axis=1, inplace = True)
    mta = mta.drop('LINENAME', axis=1)
    mta = mta.drop('DIVISION', axis=1)
    mta = mta.drop('DESC', axis=1)
    mta.columns = mta.columns.str.strip()
    return mta

In [230]:
mta1 = Col_Con(df1)
mta2 = Col_Con(df2)
mta3 = Col_Con(df3)

In [231]:
leng = len(mta1.index)
leng3 = len(mta2.index)
leng4 = len(mta3.index)

In [232]:
# make new columns, blank
#Only use if we are using turnstiles as data 'mta['Cu_ENT'] = [0] * leng'
def new_cols(mta, leng):
    mta['ENT_COUNT'] = [0] * leng
    mta['EXT_COUNT'] = [0] * leng
    mta['DATE_TIME'] = [''] * leng
    mta['DATE_TIME_WD'] = [''] * leng
    mta['Donor_Est'] = [0] * leng
    return mta

In [233]:
mta1 = new_cols(mta1, leng)
mta2 = new_cols(mta2, leng3)
mta3 = new_cols(mta3, leng4)

In [234]:
# Fill empty date time column with formula
# I dont think we will use this...
def date_time(df):
    week   = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
    for i in range(leng):
        datetime = df.DATE[i]+ ' ' + df.TIME[i]      
        value = dateutil.parser.parse(datetime)
        df.at[i,'DATE_TIME'] = value
        df.at[i,'DATE_TIME_WD'] = int(value.weekday())
    return df

In [235]:
mta1 = date_time(mta1)
mta2 = date_time(mta2)
mta3 = date_time(mta3)

In [236]:
#fill empty ent_count with formula
def coun_ent(mta, leng):
    for i in range(leng):
        if i == 0: 
            mta.at[i,'ENT_COUNT'] = 0
        else:
            if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
                diff = mta.ENTRIES[i] - mta.ENTRIES[i-1] 
                if diff < 0: diff = 0
                if diff > 10000: diff = 0
                mta.at[i,'ENT_COUNT'] = diff
            else:
                mta.at[i,'ENT_COUNT'] = 0
    return mta

In [237]:
mta1 = coun_ent(mta1,leng)
mta2 = coun_ent(mta2, leng3)
mta3 = coun_ent(mta3, leng4)

In [238]:
#fill in new exit counts
def coun_ext(mta, leng):
    for i in range(leng):
        if i == 0: 
            mta.at[i,'EXT_COUNT'] = 0
        else:
            if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
                diff = mta.EXITS[i] - mta.EXITS[i-1]
                if diff < 0: diff = 0
                if diff > 10000: diff = 0
                mta.at[i,'EXT_COUNT'] = diff
            else:
                mta.at[i,'EXT_COUNT'] = 0
                
    return mta

In [239]:
mta1 = coun_ext(mta1, leng)
mta2 = coun_ext(mta2, leng)
mta3 = coun_ext(mta3, leng)

In [240]:
mta1.head(10)

,STATION,DATE,TIME,ENTRIES,EXITS,TURNSTILE,ENT_COUNT,EXT_COUNT,DATE_TIME,DATE_TIME_WD,Donor_Est
0,59 ST,09/30/2017,00:00:00,6346858,2147624,A002R05102-00-00,0,0,2017-09-30 00:00:00,5,0
1,59 ST,09/30/2017,04:00:00,6346893,2147630,A002R05102-00-00,35,6,2017-09-30 04:00:00,5,0
2,59 ST,09/30/2017,08:00:00,6346912,2147655,A002R05102-00-00,19,25,2017-09-30 08:00:00,5,0
3,59 ST,09/30/2017,12:00:00,6347003,2147741,A002R05102-00-00,91,86,2017-09-30 12:00:00,5,0
4,59 ST,09/30/2017,16:00:00,6347270,2147801,A002R05102-00-00,267,60,2017-09-30 16:00:00,5,0
5,59 ST,09/30/2017,20:00:00,6347620,2147838,A002R05102-00-00,350,37,2017-09-30 20:00:00,5,0
6,59 ST,10/01/2017,00:00:00,6347820,2147853,A002R05102-00-00,200,15,2017-10-01 00:00:00,6,0
7,59 ST,10/01/2017,04:00:00,6347837,2147862,A002R05102-00-00,17,9,2017-10-01 04:00:00,6,0
8,59 ST,10/01/2017,08:00:00,6347847,2147880,A002R05102-00-00,10,18,2017-10-01 08:00:00,6,0
9,59 ST,10/01/2017,12:00:00,6347940,2147937,A002R05102-00-00,93,57,2017-10-01 12:00:00,6,0


In [241]:
def mor_cln(mta):   
    mta = mta.drop('TIME', axis=1)
    mta = mta.drop('ENTRIES', axis=1)
    mta = mta.drop('EXITS', axis=1)
    return mta

In [242]:
mta1 = mor_cln(mta1)
mta2 = mor_cln(mta2)
mta3 = mor_cln(mta3)

In [243]:
#tur_uni = mta.TURNSTILE.unique()

In [244]:
"""# Find some time of day trend?
turn = ''
date = ''
sum_ent = 0
cu_list = []
for i in range(leng):
    #Initialing complex if, goal is to have the cumulative for each day 
    if turn != mta.TURNSTILE[i] and i==0:
        turn = mta.TURNSTILE[i]
        date = mta.DATE[i]
        beg_in = i
        sum_ent= 0
    #if the turnstile and date are the same, sum up the ent_count
    elif (turn == mta.TURNSTILE[i] and date == mta.DATE[i]):
        sum_ent += mta['ENT_COUNT'][i]
    #if either the turnstile number changes, or the date changes, dump the sum of entries, 
    #and store in a new column across multpile indexes
    elif turn != mta.TURNSTILE[i] or date != df.DATE[i]:
        for j in range(i-beg_in):
            j +=beg_in
            mta.at[j, 'Cu_ENT'] = sum_ent 
        turn = mta.TURNSTILE[i]
        date = mta.DATE[i]
        beg_in = i
        sum_ent = mta['ENT_COUNT'][i]
mta"""

"# Find some time of day trend?\nturn = ''\ndate = ''\nsum_ent = 0\ncu_list = []\nfor i in range(leng):\n    #Initialing complex if, goal is to have the cumulative for each day \n    if turn != mta.TURNSTILE[i] and i==0:\n        turn = mta.TURNSTILE[i]\n        date = mta.DATE[i]\n        beg_in = i\n        sum_ent= 0\n    #if the turnstile and date are the same, sum up the ent_count\n    elif (turn == mta.TURNSTILE[i] and date == mta.DATE[i]):\n        sum_ent += mta['ENT_COUNT'][i]\n    #if either the turnstile number changes, or the date changes, dump the sum of entries, \n    #and store in a new column across multpile indexes\n    elif turn != mta.TURNSTILE[i] or date != df.DATE[i]:\n        for j in range(i-beg_in):\n            j +=beg_in\n            mta.at[j, 'Cu_ENT'] = sum_ent \n        turn = mta.TURNSTILE[i]\n        date = mta.DATE[i]\n        beg_in = i\n        sum_ent = mta['ENT_COUNT'][i]\nmta"

In [245]:
#Creating a function to merge aggregate columns with original DF
def merge_agg(ent_agg, ext_agg):
    ent_agg.columns = ent_agg.columns.droplevel(level=1)
    ext_agg.columns = ext_agg.columns.droplevel(level=1)
    #Creating a new data frame removing repeted values
    q_mta = pd.merge(ent_agg, ext_agg, on=['STATION','DATE_TIME_WD','DATE_TIME'], how='left')
    return q_mta

In [246]:
#Creating a function to get aggregate sum data for two columns
def get_agg(mta):
    mt = mta.groupby(['STATION','DATE_TIME_WD', 'DATE_TIME'])
    ent_agg = mt.agg({'ENT_COUNT':['sum']})
    ent_agg.reset_index(inplace=True)
    ext_agg = mt.agg({'EXT_COUNT':['sum']})
    #returns "index" as columns .reset_index()
    ext_agg.reset_index(inplace=True)
    fin_agg = merge_agg(ent_agg, ext_agg)
    return fin_agg

In [247]:
#Creating a function to get aggregate mean data for two columns
def get_agg_mean(mta):
    mt = mta.groupby(['STATION','DATE_TIME_WD', 'DATE_TIME'])
    ent_agg = mt.agg({'ENT_COUNT':['mean']})
    ent_agg.reset_index(inplace=True)
    ext_agg = mt.agg({'EXT_COUNT':['mean']})
    #returns "index" as columns .reset_index()
    ext_agg.reset_index(inplace=True)
    fin_agg = merge_agg(ent_agg, ext_agg)
    return fin_agg

In [248]:
#Updating DF with aggregate data
mta1 = get_agg(mta1)
mta2 = get_agg(mta2)
mta3 = get_agg(mta3)

In [249]:
#Function that combines 3 dfs into one
def super_merge(a,b,c):
    new_df = pd.concat([a, b])
    q_mta = pd.concat([new_df, c])
    q_mta=get_agg_mean(q_mta)
    return q_mta


In [250]:
#Combining DFs
super_df = super_merge(mta1,mta2,mta3)

In [251]:
super_df

,STATION,DATE_TIME_WD,DATE_TIME,ENT_COUNT,EXT_COUNT
0,1 AV,0,2017-08-28 00:00:00,2427,2387
1,1 AV,0,2017-08-28 04:00:00,405,339
2,1 AV,0,2017-08-28 08:00:00,1219,2634
3,1 AV,0,2017-08-28 12:00:00,4183,4305
4,1 AV,0,2017-08-28 16:00:00,2946,3289
5,1 AV,0,2017-08-28 20:00:00,5660,5308
6,1 AV,0,2017-10-02 00:00:00,2440,2247
7,1 AV,0,2017-10-02 04:00:00,403,305
8,1 AV,0,2017-10-02 08:00:00,1642,3617
9,1 AV,0,2017-10-02 12:00:00,4633,5891


In [252]:
#Creating new functions to get aggregates over an entire day and combine with original DF
def merge_agg_day(ent_agg, ext_agg):
    ent_agg.columns = ent_agg.columns.droplevel(level=1)
    ext_agg.columns = ext_agg.columns.droplevel(level=1)
    #Creating a new data frame removing repeted values
    q_mta = pd.merge(ent_agg, ext_agg, on=['STATION','DATE_TIME_WD'], how='left')
    return q_mta

def get_agg_day(mta):
    mt = mta.groupby(['STATION','DATE_TIME_WD'])
    ent_agg = mt.agg({'ENT_COUNT':['sum']})
    ent_agg.reset_index(inplace=True)
    ext_agg = mt.agg({'EXT_COUNT':['sum']})
    #returns "index" as columns .reset_index()
    ext_agg.reset_index(inplace=True)
    fin_agg = merge_agg_day(ent_agg, ext_agg)
    return fin_agg

In [253]:
#Creates a DF with cumulative entries per day per station with no datetime object
no_dt = get_agg_day(super_df)
no_dt

,STATION,DATE_TIME_WD,ENT_COUNT,EXT_COUNT
0,1 AV,0,58858,63039
1,1 AV,1,62236,66110
2,1 AV,2,64197,66631
3,1 AV,3,65635,67611
4,1 AV,4,67502,69919
5,1 AV,5,38326,42913
6,1 AV,6,44181,50110
7,103 ST,0,91787,60558
8,103 ST,1,95865,64722
9,103 ST,2,97221,65457


In [254]:
"""from datetime import datetime as dt
noon = dateutil.parser.parse('12:00:00')
mask = (super_df["DATE_TIME"].dt.time >= noon.time())
#making a new df with only afternoon values
no_morn = super_df[mask]
"""

'from datetime import datetime as dt\nnoon = dateutil.parser.parse(\'12:00:00\')\nmask = (super_df["DATE_TIME"].dt.time >= noon.time())\n#making a new df with only afternoon values\nno_morn = super_df[mask]\n'

In [255]:
#Using online stats to calculate est donor volume from entries
def add_donors(df):
    leng=len(df.index)
    for i in range(leng):
        val= round(df['ENT_COUNT'][i]*.0005)
        df.at[i,'Donor_Est'] = val
    return df

In [316]:
#Sorting DF and getting the top 30 stations
super_df.sort_values(by=['ENT_COUNT'], ascending = 0, inplace = True)
uni_stations = list(super_df.STATION.unique())[:20]
uni_stations

['34 ST-HERALD SQ',
 'TIMES SQ-42 ST',
 '14 ST-UNION SQ',
 '47-50 STS ROCK',
 '34 ST-PENN STA',
 '23 ST',
 'GRD CNTRL-42 ST',
 'CHAMBERS ST',
 '59 ST COLUMBUS',
 '59 ST',
 'FULTON ST',
 '42 ST-BRYANT PK',
 'JKSN HT-ROOSVLT',
 '86 ST',
 '50 ST',
 'LEXINGTON AV/53',
 'CANAL ST',
 '42 ST-PORT AUTH',
 'JAY ST-METROTEC',
 'FLUSHING-MAIN']

In [317]:
super_df.reset_index(drop=True, inplace = True)
f = dateutil.parser.parse('04:00:00')
late = dateutil.parser.parse('22:00:00')
#Creating a better df for reading traffic per date tiem
for i in range(len(super_df.index)):
    val = (super_df['DATE_TIME'][i]).hour
    super_df.at[i,'New_DATE_TIME'] = val
    if super_df.ENT_COUNT[i] >=100000 and (val<f.time.hour() or val>late.time.hour()):
        super_df.drop(i, inplace = True)
    if super_df.ENT_COUNT[i] <=10:
        super_df.drop(i, inplace = True)

In [307]:
#Creating new data while sorting out low traffic stations, then resetting the indeces
uni_df = super_df[super_df['STATION'].isin(uni_stations)]
uni_df.reset_index(drop=True, inplace = True)


In [308]:
#adding donor col to uni_df
uni_df = add_donors(uni_df)


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [309]:
uni_df

,STATION,DATE_TIME_WD,DATE_TIME,ENT_COUNT,EXT_COUNT,New_DATE_TIME,Donor_Est
0,34 ST-HERALD SQ,1,2017-10-17 20:00:00,49595,32266,20,25.0
1,34 ST-HERALD SQ,0,2017-10-16 20:00:00,49471,31518,20,25.0
2,34 ST-HERALD SQ,2,2017-10-18 20:00:00,49289,31243,20,25.0
3,34 ST-HERALD SQ,3,2017-10-19 20:00:00,48513,33482,20,24.0
4,34 ST-HERALD SQ,2,2017-10-04 20:00:00,47830,31056,20,24.0
5,34 ST-HERALD SQ,3,2017-10-05 20:00:00,47797,32246,20,24.0
6,34 ST-HERALD SQ,1,2017-08-29 20:00:00,47682,32514,20,24.0
7,34 ST-HERALD SQ,0,2017-08-28 20:00:00,46861,29612,20,23.0
8,34 ST-HERALD SQ,2,2017-08-30 20:00:00,46797,32664,20,23.0
9,34 ST-HERALD SQ,3,2017-08-31 20:00:00,46627,30752,20,23.0


In [310]:
#Creating new functions to get aggregates over an entire day and combine with original DF
def merge_agg_time(ent_agg, ext_agg):
    ent_agg.columns = ent_agg.columns.droplevel(level=1)
    ext_agg.columns = ext_agg.columns.droplevel(level=1)
    #Creating a new data frame removing repeted values
    q_mta = pd.merge(ent_agg, ext_agg, on=['STATION','New_DATE_TIME'], how='left')
    return q_mta

def get_agg_time(mta):
    mt = mta.groupby(['STATION','New_DATE_TIME'])
    ent_agg = mt.agg({'ENT_COUNT':['sum']})
    ent_agg.reset_index(inplace=True)
    ext_agg = mt.agg({'EXT_COUNT':['sum']})
    #returns "index" as columns .reset_index()
    ext_agg.reset_index(inplace=True)
    fin_agg = merge_agg_time(ent_agg, ext_agg)
    return fin_agg


In [311]:
uni_df = get_agg_time(uni_df)
#uni_df.drop(0)
uni_df

,STATION,New_DATE_TIME,ENT_COUNT,EXT_COUNT
0,14 ST-UNION SQ,5,36417,18103
1,14 ST-UNION SQ,7,2716,6272
2,14 ST-UNION SQ,8,3705,7462
3,14 ST-UNION SQ,9,130921,303583
4,14 ST-UNION SQ,10,2738,7754
5,14 ST-UNION SQ,11,2363,2815
6,14 ST-UNION SQ,13,264971,475116
7,14 ST-UNION SQ,14,7898,6069
8,14 ST-UNION SQ,17,563035,426450
9,14 ST-UNION SQ,21,740897,409738


In [312]:
no_dt_df = no_dt[no_dt['STATION'].isin(uni_stations)]
no_dt_df.sort_values(by='ENT_COUNT', ascending = 0, inplace = True)
no_dt_df.reset_index(drop=True, inplace=True)
no_dt_df['Donor_Est'] = [0] * len(no_dt_df.index)
no_dt_df = add_donors(no_dt_df)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [313]:
no_dt_df

,STATION,DATE_TIME_WD,ENT_COUNT,EXT_COUNT,Donor_Est
0,34 ST-PENN STA,1,497928,420358,249
1,34 ST-PENN STA,2,496567,415773,248
2,34 ST-PENN STA,3,493921,432339,247
3,34 ST-PENN STA,0,479452,393976,240
4,34 ST-PENN STA,4,475932,430725,238
5,GRD CNTRL-42 ST,1,471618,438115,236
6,GRD CNTRL-42 ST,2,457910,433538,229
7,GRD CNTRL-42 ST,3,452682,430439,226
8,GRD CNTRL-42 ST,0,432193,409126,216
9,GRD CNTRL-42 ST,4,415939,399704,208


In [314]:
# Next step! Combine days together and analyze what locations are the most popular and one what days
import seaborn as sns
import matplotlib.dates as mdates

# build the figure
%matplotlib osx

fig, ax = plt.subplots(figsize=(15,12))
fig = sns.pointplot(y="ENT_COUNT", x="New_DATE_TIME", data=uni_df, join=True, hue='STATION', ax=ax, linestyles='-').set_title('10 Largest Stations: Daily Traffic')
ax.set(xlabel='Time', ylabel='Total Entries')

[Text(0,0.5,'Total Entries'), Text(0.5,0,'Time')]

In [ ]:
%matplotlib osx
week_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
week = ['Monday','Tuesday', 'Wednesday','Thursday', 'Friday','Saturday', 'Sunday']
fig, ax = plt.subplots(figsize=(15,12))
fig = sns.pointplot(y="ENT_COUNT", x="DATE_TIME_WD", data=no_dt_df, join=True, hue='STATION', ax=ax, linestyles='-').set_title('10 Largest Traffic Stations')
ax.set(xlabel='Dates', ylabel='Total Entries', xticklabels=week)


#### Challenge 2

- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}



#### Challenge 3

- These counts are for every n hours. (What is n?) We want total daily
  entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.


#### Challenge 4

We will plot the daily time series for a turnstile.

In ipython notebook, add this to the beginning of your next cell:

    %matplotlib inline

This will make your matplotlib graphs integrate nicely with the
notebook. To plot the time series, import matplotlib with

    import matplotlib.pyplot as plt

Take the list of [(date1, count1), (date2, count2), ...], for the
turnstile and turn it into two lists:
dates and counts. This should plot it:

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)
    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#### Challenge 5

- So far we've been operating on a single turnstile level, let's
  combine turnstiles in the same ControlArea/Unit/Station combo. There
  are some ControlArea/Unit/Station groups that have a single
  turnstile, but most have multiple turnstilea-- same value for the
  C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together -- for each
ControlArea/UNIT/STATION combo, for each day, add the counts from each
turnstile belonging to that combo.

#### Challenge 6

Similarly, combine everything in each station, and come up with a time
series of `[(date1, count1),(date2,count2),...]` type of time series
for each STATION, by adding up all the turnstiles in a station.

#### Challenge 7

Plot the time series for a station.

#### Challenge 8

- Make one list of counts for **one** week for one station. Monday's
count, Tuesday's count, etc. so it's a list of 7 counts.
Make the same list for another week, and another week, and another
week.
`plt.plot(week_count_list)` for every `week_count_list` you created
this way. You should get a rainbow plot of weekly commute numbers on
top of each other.

#### Challenge 9

- Over multiple weeks, sum total ridership for each station and sort
  them, so you can find out the stations with the highest traffic
  during the time you investigate

#### Challenge 10

- Make a single list of these total ridership values and plot it with

    plt.hist(total_ridership_counts)

to get an idea about the distribution of total ridership among
different stations.

This should show you that most stations have a small traffic, and the
histogram bins for large traffic volumes have small bars.

*Additional Hint*:

If you want to see which stations take the meat of the traffic, you
can sort the total ridership counts and make a `plt.bar` graph. For
this, you want to have two lists: the indices of each bar, and the
values. The indices can just be `0,1,2,3,...`, so you can do

    indices = range(len(total_ridership_values))
    plt.bar(indices, total_ridership_values)